### Задание № 1

In [2]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [1]:
import gensim
import numpy as np
import pandas as pd
from datetime import datetime
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from string import punctuation
from razdel import tokenize as razdel_tokenize
import pickle
import warnings
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

In [2]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [3]:
data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)

In [4]:
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, max_df=0.4)
X = vectorizer.fit_transform(data['description_norm'])

In [5]:
pipelines = {
    'svd_sgd' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))]),
        
    'svd_kn' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier(n_neighbors=7))]),
        
    'svd_rf' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))]),
        
    'svd_extratrees' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier(random_state=0))]),
   
    'nmf_sgd' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))]),

    'nmf_kn' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', KNeighborsClassifier(n_neighbors=7))]),
        
    'nmf_rf' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))]),

    'nmf_extratrees' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', ExtraTreesClassifier(random_state=0))]),
}

In [6]:
def eval_table(X, y, pipeline, N=6):
    labels = list(set(y))
 
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))

    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])

        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')

    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
 
    result.loc['mean'] = result.mean().round(2)
    errors /= N
    
    return result, errors

In [7]:
for count, pipe in enumerate(pipelines):
    metrics_svd, errors_svd = eval_table(data['description_norm'], data['category_name'], pipelines[pipe])
    print(f'{pipe}: {metrics_svd.loc["mean"]["f1"].round(3)}\n', end='')

svd_sgd: 0.76
svd_kn: 0.47
svd_rf: 0.56
svd_extratrees: 0.62
nmf_sgd: 0.52
nmf_kn: 0.52
nmf_rf: 0.62
nmf_extratrees: 0.7


**лучшее сочетание - svd_sgd**

### Задание № 2 

In [8]:
texts = open('wiki_data.txt', encoding='utf-8').read().splitlines()[:5000]

In [9]:
texts = ([normalize(text) for text in texts])

In [10]:
text = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [11]:
dictinary = gensim.corpora.Dictionary((text.split() for text in texts))
dictinary.filter_extremes(no_above=0.1, no_below=10)
dictinary.compactify()

print(dictinary)

Dictionary(8116 unique tokens: ['1,2', '1,5', '12', '14', '16']...)


In [12]:
corpus = [dictinary.doc2bow(text.split()) for text in texts]
lda = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10)
lda.print_topics()

[(99,
  '0.020*"писатель" + 0.020*"партия" + 0.018*"литературный" + 0.017*"союз" + 0.017*"италия" + 0.014*"итальянский" + 0.013*"фестиваль" + 0.011*"одесский" + 0.011*"журнал" + 0.009*"международный"'),
 (97,
  '0.012*"н" + 0.012*"церковь" + 0.011*"пуля" + 0.009*"николай" + 0.006*"иванович" + 0.005*"жизнь" + 0.004*"ботанический" + 0.004*"играть" + 0.004*"памятник" + 0.004*"я"'),
 (98,
  '0.019*"император" + 0.017*"армия" + 0.014*"войско" + 0.014*"китай" + 0.012*"римский" + 0.011*"провинция" + 0.010*"битва" + 0.010*"империя" + 0.009*"ван" + 0.008*"против"'),
 (96,
  '0.017*"мм" + 0.015*"растение" + 0.015*"длина" + 0.012*"смотреть" + 0.009*"орудие" + 0.009*"десант" + 0.008*"лист" + 0.007*"ствол" + 0.007*"ширина" + 0.006*"семейство"'),
 (94,
  '0.019*"клетка" + 0.019*"белок" + 0.016*"схема" + 0.014*"форма" + 0.013*"канада" + 0.011*"промежуточный" + 0.008*"экипаж" + 0.008*"малый" + 0.008*"структура" + 0.007*"распределять"'),
 (95,
  '0.039*"доктор" + 0.010*"больница" + 0.010*"мозг" + 0.009

In [13]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

5374.3030950264965

In [14]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in texts], 
                                                   dictionary=dictinary, coherence='c_v')
coherence_model_lda.get_coherence()

0.5405740625275799

In [15]:
[text for text in ngrammed_texts[:3]]

['новостройка нижегородский область новостро́йка — сельский посёлок в дивеевский район нижегородский область входить в состав сатисский сельсовет посёлок расположить в 12,5 км к юг от село дивеево и 1 км к запад от город саров на право берег река вичкинза правый приток река сатис окружить смешанный лес соединить асфальтовый дорога с посёлок цыгановка 1,5 км и грунтовый просёлочный дорога с посёлок сатис 3,5 км название новостройка являться сугубо официальный местный население использовать исключительно альтернативный название — хитрый употребляться языковой оборот « … на хитрый » ранее использовать название — песчаный известковый основать в 1920-й год переселенец из соседний село аламасовый и нарышкино расположить соответственно в 8 и 14 км к запад в вознесенский район традиционно в посёлок жить рабочий совхоз « вперёд » центр в посёлок сатис возле посёлок расположить карьер где активно добывать доломитовый мука и бутовый камень в настоящий время официально закрытый по данные 1978 год 

## Ngrammed 

In [16]:
ngrammed_dictinary = gensim.corpora.Dictionary((text.split() for text in ngrammed_texts))
ngrammed_dictinary.filter_extremes(no_above=0.05, no_below=10)
ngrammed_dictinary.compactify()

In [21]:
ngrammed_corpus = [ngrammed_dictinary.doc2bow(text.split()) for text in ngrammed_texts]

In [22]:
ngrammed_lda = gensim.models.LdaMulticore(ngrammed_corpus, 
                                 100,
                                 alpha='asymmetric',
                                 id2word=ngrammed_dictinary, 
                                 passes=10) 

In [23]:
ngrammed_lda.print_topics()

[(99,
  '0.016*"симфония" + 0.013*"деревня" + 0.010*"устройство" + 0.010*"муниципальный" + 0.010*"самара" + 0.009*"самарский" + 0.008*"тверской" + 0.008*"пермский" + 0.008*"дума" + 0.008*"чайковский"'),
 (98,
  '0.035*"значение" + 0.014*"1996" + 0.014*"формула" + 0.012*"вагон" + 0.011*"прыжок" + 0.011*"функция" + 0.010*"formula_2" + 0.010*"волна" + 0.009*"formula_3" + 0.008*"formula_1"'),
 (97,
  '0.012*"клиент" + 0.011*"дворец" + 0.007*"ла" + 0.006*"помещение" + 0.006*"издание" + 0.005*"мода" + 0.005*"статья" + 0.005*"речь" + 0.005*"база" + 0.004*"зал"'),
 (96,
  '0.037*"экспедиция" + 0.018*"полюс" + 0.018*"норвежский" + 0.017*"собака" + 0.012*"полярный" + 0.011*"норвегия" + 0.011*"скотт" + 0.011*"широта" + 0.010*"100" + 0.010*"план"'),
 (94,
  '0.021*"’" + 0.013*"собака" + 0.010*"чёрный" + 0.007*"говорить" + 0.006*"я" + 0.005*"порода" + 0.005*"деньга" + 0.005*"робот" + 0.005*"двор" + 0.005*"друг"'),
 (95,
  '0.020*"святой" + 0.009*"мария" + 0.009*"деревня" + 0.007*"церковь" + 0.006*"

In [33]:
np.exp2(-ngrammed_lda.log_perplexity(corpus[:1000]))

18632.197714724345

In [26]:
coherence_model_ngrammed_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in ngrammed_texts], 
                                                   dictionary=ngrammed_dictinary, coherence='c_v')
coherence_model_ngrammed_lda.get_coherence()

0.5562539933221263

## TfIdf

In [27]:
dictinary = gensim.corpora.Dictionary((text.split() for text in texts))
dictinary.filter_extremes(no_above=0.1, no_below=10)
dictinary.compactify()

print(dictinary)

Dictionary(8116 unique tokens: ['1,2', '1,5', '12', '14', '16']...)


In [28]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary, )
corpus_tfidf = tfidf[corpus]

In [29]:
lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                 100,
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10) 

In [30]:
lda_tfidf.print_topics()

[(99,
  '0.011*"мухаммад" + 0.000*"б" + 0.000*"посёлок" + 0.000*"партия" + 0.000*"бомба" + 0.000*"рекомендоваться" + 0.000*"сантиметр" + 0.000*"николаевский" + 0.000*"альпы" + 0.000*"восстанавливаться"'),
 (97,
  '0.000*"противоречие" + 0.000*"публично" + 0.000*"признанный" + 0.000*"принудительный" + 0.000*"прогресс" + 0.000*"продажа" + 0.000*"последователь" + 0.000*"противоречить" + 0.000*"согласный" + 0.000*"расстрел"'),
 (98,
  '0.014*"71" + 0.000*"девиз" + 0.000*"библия" + 0.000*"стих" + 0.000*"монастырь" + 0.000*"орден" + 0.000*"море" + 0.000*"72" + 0.000*"собор" + 0.000*"андреевич"'),
 (95,
  '0.004*"иностранец" + 0.003*"китаец" + 0.000*"хуан" + 0.000*"польский" + 0.000*"китай" + 0.000*"гитлер" + 0.000*"польша" + 0.000*"вступать" + 0.000*"11-й" + 0.000*"путин"'),
 (96,
  '0.010*"цивилизация" + 0.004*"разум" + 0.000*"контакт" + 0.000*"разумный" + 0.000*"человечество" + 0.000*"сообщение" + 0.000*"планета" + 0.000*"активный" + 0.000*"прогресс" + 0.000*"оценка"'),
 (94,
  '0.000*"кан

In [34]:
np.exp2(-lda_tfidf.log_perplexity(corpus[:1000]))

2307.390166311586

In [36]:
coherence_model_lda_tfidf = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in texts], 
                                                   dictionary=dictinary, coherence='c_v')
coherence_model_lda_tfidf.get_coherence()

0.5405740625275799

## модель с нграммами и tfidf

In [37]:
ngrammed_tfidf = gensim.models.TfidfModel(ngrammed_corpus, id2word=ngrammed_dictinary, )
corpus_ngrammed_tfidf = tfidf[ngrammed_corpus]

In [38]:
lda_ngrammed_tfidf = gensim.models.LdaMulticore(corpus_ngrammed_tfidf, 
                                 100,
                                 alpha='asymmetric',
                                 id2word=ngrammed_dictinary, 
                                 passes=2)

In [39]:
lda_ngrammed_tfidf.print_topics()

[(99,
  '0.000*"почему" + 0.000*"предъявить" + 0.000*"подписание" + 0.000*"подчеркнуть" + 0.000*"понимание" + 0.000*"постель" + 0.000*"переселение" + 0.000*"представительство" + 0.000*"рак" + 0.000*"проявить"'),
 (98,
  '0.018*"коми" + 0.003*"республика" + 0.002*"вячеслав" + 0.002*"юар" + 0.001*"глава" + 0.001*"арест" + 0.001*"суд" + 0.001*"следствие" + 0.001*"2015" + 0.001*"я"'),
 (97,
  '0.004*"тысяча" + 0.002*"тыс" + 0.002*"uss" + 0.002*"республика" + 0.002*"1965" + 0.002*"численность" + 0.002*"конфликт" + 0.001*"житель" + 0.001*"океан" + 0.001*"2015"'),
 (96,
  '0.003*"конкурс" + 0.002*"ван" + 0.002*"житель" + 0.002*"музыкант" + 0.002*"пианист" + 0.002*"московский" + 0.001*"чайковский" + 0.001*"церковь" + 0.001*"воронеж" + 0.001*"техас"'),
 (95,
  '0.007*"норвежский" + 0.002*"пьеса" + 0.002*"искусство" + 0.002*"перевод" + 0.001*"детский" + 0.001*"осло" + 0.001*"художник" + 0.001*"перевести" + 0.001*"премия" + 0.001*"известность"'),
 (93,
  '0.000*"почему" + 0.000*"предъявить" + 0.0

In [40]:
np.exp2(-lda.log_perplexity(corpus_ngrammed_tfidf[:1000]))

2.418510199058425e+41

In [42]:
coherence_model_lda_ngrammed_tfidf = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in ngrammed_texts], 
                                                   dictionary=ngrammed_dictinary, coherence='c_v')
coherence_model_lda_ngrammed_tfidf.get_coherence()

0.5562539933221263

## Анализ результатов
Оценивая модели на глаз, мне кажется, лучше всего с подбором тем справляется обычная модель. 

Самая красивая для меня тема в ней: **(92,'0.014*"книга" + 0.012*"штат" + 0.008*"ключ" + 0.008*"танец" + 0.008*"американский" + 0.007*"балет" + 0.007*"если" + 0.006*"издание" + 0.006*"матрица" + 0.006*"точка"')**

В нграммной модели: **(4,'0.044*"театр" + 0.015*"актёр" + 0.012*"режиссёр" + 0.012*"кино" + 0.011*"театральный" + 0.011*"актриса" + 0.011*"спектакль" + 0.010*"искусство" + 0.008*"драматический" + 0.008*"сцена"')**

В tfidf: **(96,'0.010*"цивилизация" + 0.004*"разум" + 0.000*"контакт" + 0.000*"разумный" + 0.000*"человечество" + 0.000*"сообщение" + 0.000*"планета" + 0.000*"активный" + 0.000*"прогресс" + 0.000*"оценка"')**

В ngrammed_tfidf: **(92,'0.002*"тюркский" + 0.002*"китай" + 0.001*"правление" + 0.001*"хан" + 0.001*"ставка" + 0.001*"император" + 0.001*"империя" + 0.001*"выдать" + 0.001*"государство" + 0.001*"источник"')**


### Перплексия
Ближе всего к нулю перплексия модели с нграммами и tfidf, худшиу показатьль у нграммной модели.

### Когерентность

В целом, все модели выдают примерно одинаковые результаты, однако с подбором тем лучше всего справляются обычная и нграммная модели.